In [111]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os.path
from scipy import optimize

In [116]:
cs_T_adj = 0.9  # finite temp
rb_T_adj = 0.86  # finite temp

cs_freqs = np.array([
   [55, 0.70162788],
   [50.65, 0.9084147],
   [39.44, 1.82411816],
   [26.77, 3.20483396],
   [17.65, 3.8410576],
   [9.219, 4.04551988]
])*np.array([0.81*1.05*cs_T_adj,1]) + np.array([0, -4.4742177])

rb_freqs = np.array([
   [55, 0.0849255],
   [50.65, 0.10782397],
   [39.44, 0.53102993],
   [26.77, 0.79220492],
   [17.65, 1.02125303],
   [9.219, 1.1057729]
   ])*np.array([0.81*1.05*rb_T_adj,1]) + np.array([0, -0.64949647])

bz = 1.15 # magnetic field in G
cs_qz = 4.2745e-1*bz**2 # quadratic zeeman (kHz) Steck
rb_qz = 5.7515e-1*bz**2 # quadratic zeeman (kHz) Steck

cs_diff = -0.07913  # differential stark shift theory, kHz/mW at 1064
cs_diff_u = cs_diff*0.95**2  # 5% waist error
cs_diff_l = cs_diff*1.05**2


rb_diff = -0.021185  # differential stark shift theory, kHz/mW at 1064
rb_diff_u = rb_diff*0.95**2
rb_diff_l = rb_diff*1.05**2

In [117]:
cserr = np.array([
  0.187,
  0.154,
  0.108,
  0.235,
  0.12,
  0.18
])
rberr = np.array([
  0.215,
  0.27,
  0.166,
  0.243,
  0.163,
  0.2
])

In [118]:
def alpha_filter(c, a):
    # c:color, a:alpha
#     print(mpl.colors.cnames[c])
#     print([mpl.colors.cnames[c][1:][i:i+2] for i in range(3)])
    src = np.array([int(mpl.colors.cnames[c][1:][i:i+2],16)/255. for i in range(0,6,2)])
    tar = (1-a) + src*a
    return tar

alpha_filter('indianred', 1)

array([0.80392157, 0.36078431, 0.36078431])

In [119]:
fig, ax = plt.subplots(figsize=(2*3.375,3.5))
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None', markersize=6),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None', markersize=6)
]
xs = np.linspace(0,60,2)

r=0
cs_fit = np.polyfit(cs_freqs[:,0], cs_freqs[:,1]-cs_qz, 1, w=cserr**-1, cov=True)

def lin(x, s):
    return x*s

print '---cs---'
popt, pcov = optimize.curve_fit(lin, cs_freqs[:,0], cs_freqs[:,1]-cs_qz, sigma=cserr)
print(popt)
print(np.sqrt(np.diagonal(np.abs(pcov))))
a = popt[0]

# print('{:.4f}'.format(a[0]))

# print cs_fit[0]
# print np.sqrt(np.abs(np.diag(cs_fit[1])))

ax.errorbar(cs_freqs[:,0], cs_freqs[:,1]-cs_qz,
        yerr=cserr,
        **marker_styles[r]
    )
# ax.plot(xs, np.poly1d([a,0])(xs), color='cornflowerblue', linestyle='-')
ax.plot(xs, np.poly1d([cs_diff, 0])(xs), color='cornflowerblue', linestyle='-')
ax.fill_between(xs, np.poly1d([cs_diff_u, 0])(xs), y2=np.poly1d([cs_diff_l, 0])(xs), color=alpha_filter('cornflowerblue', 0.3), linestyle='None')

print '---rb---'
r=1
rb_fit = np.polyfit(rb_freqs[:,0], rb_freqs[:,1]-rb_qz, 1, w=rberr**-1, cov=True)

popt, pcov = optimize.curve_fit(lin, rb_freqs[:,0], rb_freqs[:,1]-rb_qz, sigma=rberr)
print(popt)
print(np.sqrt(np.diagonal(np.abs(pcov))))
b = popt[0]

# print rb_fit[0]
# print np.sqrt(np.abs(np.diag(rb_fit[1])))

ax.errorbar(rb_freqs[:,0], rb_freqs[:,1]-rb_qz,
        yerr=rberr,
        **marker_styles[r]
    )

# ax.plot(xs, np.poly1d([b,0])(xs), color='indianred', linestyle='-')

ax.fill_between(xs, np.poly1d([rb_diff_u, 0])(xs), y2=np.poly1d([rb_diff_l, 0])(xs), color=alpha_filter('indianred', 0.3), linestyle='None')
ax.plot(xs, np.poly1d([rb_diff, 0])(xs), color='indianred', linestyle='-')

ax.grid(which='both')
ax.set_ylabel(r'$f- f_{hf}(B)$ (kHz)', fontsize=14)
ax.set_xlabel(r'$P_{ODT}$ (mW)', fontsize=14)
ax.set_xlim(0,45)
plt.tight_layout()

fmts = ['pdf', 'png', 'eps']
out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/diff-trap-shift/'
fn = os.path.join(out, 'trap-shift_corrected.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

---cs---
[-0.10428942]
[0.0026609]
---rb---
[-0.03247522]
[0.00099618]


In [79]:
rb_qz

0.760635875

In [80]:
cs_qz

0.565302625

In [81]:
2.38*(cs_diff/a)**0.5

2.408416895552809

In [82]:
2.38*(rb_diff/b)**0.5

2.2845188757144244

In [83]:
rb_diff

-0.0218857

In [86]:
mV2mW = 0.523822

In [87]:
90*mV2mW*a

-3.7634256318092807

In [88]:
90*mV2mW*b

-1.1198274222911606